In [1]:
import tensorflow as tf
import os

d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\develop\python\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:544: FutureWarning: Passi

In [3]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64,input_shape=(3,)))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                256       
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['acc'])

# 训练中保存模型

In [18]:
model_save = 'E:\\study\\tensorflow\\h5\\cp\\cp.ckpt'

cp_callback = tf.keras.callbacks.ModelCheckpoint(model_save,save_weights_only=True)

In [ ]:
model.fit(train,label,epoch=5,callbacks=[cp_callback])

In [ ]:
model.load_weights(model_save)

# 保存全部架构，权重，优化器

In [6]:
model.save('E:\\study\\tensorflow\\h5\\test.h5')

In [7]:
in_model = tf.keras.models.load_model('E:\\study\\tensorflow\\h5\\test.h5')

In [8]:
in_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                256       
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


# 仅保存架构

In [9]:
model_json = model.to_json()

In [10]:
model_json

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 3], "dtype": "float32", "units": 64, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "keras_version": "2.2.4-tf", "backend": "tensorflow"}'

In [11]:
reinitialized_model = tf.keras.models.model_from_json(model_json)

In [12]:
reinitialized_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                256       
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


# 仅保存权重

In [13]:
weights = model.get_weights()

weights

[array([[-0.05684707,  0.10033345,  0.18884224,  0.05521312, -0.06020766,
          0.10414413, -0.21557227,  0.18269208,  0.2500999 , -0.12600833,
          0.25455326,  0.25915825,  0.15703568,  0.16725072, -0.21538049,
          0.23405981, -0.0723094 , -0.2845269 , -0.23936078,  0.08777291,
         -0.17290187, -0.24723326, -0.23326185, -0.116258  ,  0.22826552,
         -0.05714737,  0.16580787,  0.25049424,  0.07208917,  0.02652904,
         -0.01792505,  0.20070767,  0.08313516,  0.08219808, -0.0820372 ,
         -0.17388096,  0.08389044,  0.25320596, -0.2552694 , -0.2816666 ,
         -0.18277235,  0.08745641,  0.02176747,  0.2157594 , -0.25148678,
          0.19943982,  0.23321068, -0.2964807 , -0.13700505, -0.24046053,
          0.05828628, -0.15955654, -0.02552256, -0.25979745,  0.22642654,
         -0.11260751, -0.15072045,  0.09076926,  0.28451645,  0.20416689,
         -0.19111344, -0.27442312,  0.07409853,  0.22417134],
        [ 0.14684364,  0.13167995, -0.07553053,  0

In [14]:
reinitialized_model.set_weights(weights)

In [15]:
model.save_weights('E:\\study\\tensorflow\\h5\\test_weight.h5')

In [16]:
reinitialized_model.load_weights('E:\\study\\tensorflow\\h5\\test_weight.h5')

# 自定义训练保存模型

In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32,input_shape=(3,),activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


In [3]:
optimizer = tf.keras.optimizers.Adam(0.0001) 

In [4]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [5]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)

In [6]:
def train_step(model,image,label):
    with tf.GradientTape() as t:
        pred = model(image)
        loss_step = loss_func(label,pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [7]:
def train():
    for epoch in range(10):
        for (batch,(images, labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print('Epoch{} lossis {},accuracy is {}'.format(epoch,train_loss.result(),train_accuracy.result()))
  
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()
        
        if (epoch + 1)%10 == 0:
            checkpoint.save(file_prefix=cp_prefix)

In [8]:
cp_dir = 'E:\\study\\tensorflow\\h5\\custom_cp'
cp_prefix = os.path.join(cp_dir,'ckpt')

In [9]:
checkpoint = tf.train.Checkpoint(
    optimizer=optimizer,
    model=model
)

In [ ]:
checkpoint.restore(tf.train.load_checkpoint(cp_dir))

In [ ]:
tf.argmax(model(image,training=False),axis=1)

In [ ]:
#求正确率
(tf.argmax(model(image,training=False),axis==1) == label.sum())/len(label)